## PREPROCESING

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sys
import re

BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"

def get_prodi_info(prodi_id):
    """
    Mengambil nama prodi dan jumlah halaman maksimum dalam satu kali request.
    """
    url = f"{BASE_URL}/{prodi_id}/1"
    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")

        prodi_name_element = soup.select_one('div#begin h2')
        if not prodi_name_element:
            return None, 0
        
        prodi_full = prodi_name_element.text.strip()
        prodi_name = prodi_full.replace("Journal Jurusan ", "")

        last_page_link = soup.select_one('ol.pagination a:contains("»")')
        if last_page_link and "href" in last_page_link.attrs:
            href = last_page_link["href"]
            max_page = int(href.split("/")[-1])
        else:
            max_page = 1
            
        return prodi_name, max_page
        
    except requests.exceptions.RequestException as e:
        print(f"\nError saat mengakses informasi prodi {prodi_id}: {e}")
        return None, 0

def print_progress(prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 30
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\rScraping {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()

def scrape_prodi(prodi_id):
    prodi_name, max_page = get_prodi_info(prodi_id)
    if not prodi_name:
        print(f"Gagal mendapatkan informasi untuk prodi ID: {prodi_id}. Proses dihentikan.")
        return None
        
    print(f"Prodi terdeteksi: {prodi_name} (ID: {prodi_id})")
    print(f"Ditemukan {max_page} halaman untuk di-scrape.")
    
    data = {
        "id": [], "penulis": [], "judul": [], "abstrak_id": [], "abstrak_en": [],
        "pembimbing_pertama": [], "pembimbing_kedua": [], "prodi": []
    }

    for page in range(1, max_page + 1):
        url = f"{BASE_URL}/{prodi_id}/{page}"
        try:
            r = requests.get(url)
            r.raise_for_status()
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            for jurnal in jurnals:
                link_detail = jurnal.select_one('a.gray.button')['href']
                id_match = re.search(r"/detail/(\d+)", link_detail)
                pta_id = id_match.group(1) if id_match else None

                response = requests.get(link_detail)
                soup_detail = BeautifulSoup(response.content, "html.parser")
                isi = soup_detail.select_one('div#content_journal')
                if not isi:
                    continue

                def get_metadata_value_robust(soup_content, label):
                    element = soup_content.select_one(f'span:-soup-contains("{label}")')
                    if element:
                        parent_text = element.parent.get_text(strip=True)
                        parts = parent_text.split(':', 1)
                        if len(parts) > 1:
                            return parts[1].strip()
                    return "N/A"

                judul = isi.select_one('a.title').text.strip()
                penulis = get_metadata_value_robust(isi, "Penulis")
                pembimbing_pertama = get_metadata_value_robust(isi, "Dosen Pembimbing I")
                pembimbing_kedua = get_metadata_value_robust(isi, "Dosen Pembimbing II")
                
                paragraf = isi.select('p[align="justify"]')
                abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
                abstrak_en = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"

                data["id"].append(pta_id)
                data["penulis"].append(penulis)
                data["judul"].append(judul)
                data["abstrak_id"].append(abstrak_id)
                data["abstrak_en"].append(abstrak_en)
                data["pembimbing_pertama"].append(pembimbing_pertama)
                data["pembimbing_kedua"].append(pembimbing_kedua)
                data["prodi"].append(prodi_name)

            print_progress(prodi_name, page, max_page)
        
        except requests.exceptions.RequestException as e:
            print(f"\nError pada halaman {page} untuk prodi {prodi_name}: {e}")
            continue

    if not data["id"]:
        print("\nTidak ada data yang berhasil di-scrape.")
        return None

    df = pd.DataFrame(data)
    output_filename = f"pta_{prodi_name.lower().replace(' ', '')}{prodi_id}.csv"
    df.to_csv(output_filename, index=False, encoding="utf-8-sig")
    
    print(f"\n\n✅ Selesai! Data prodi {prodi_name} berhasil dikumpulkan.")
    print(f"📄 Data disimpan dalam file: {output_filename}")

    return df

# --- Cara Menjalankan ---
if __name__ == "__main__":
    ID_PRODI_TARGET = 7
    df_hasil = scrape_prodi(ID_PRODI_TARGET)

    if df_hasil is not None:
        print("\nContoh 5 data pertama yang diambil:")
        print(df_hasil.head())


Prodi terdeteksi: Manajemen (ID: 7)
Ditemukan 207 halaman untuk di-scrape.
Scraping Manajemen - Page 207/207 [██████████████████████████████] 100.00%

✅ Selesai! Data prodi Manajemen berhasil dikumpulkan.
📄 Data disimpan dalam file: pta_manajemen7.csv

Contoh 5 data pertama yang diambil:
             id                  penulis  \
0  080211100070                  SATIYAH   
1  090211200001                  Faishal   
2  080211100050          Wahyu Kurniawan   
3  100211200002   Muhammad Zakaria Utomo   
4  080211100044  Hendri Wahyudi Prayitno   

                                               judul  \
0  PENGARUH FAKTOR-FAKTOR PELATIHAN DAN PENGEMBAN...   
1  ANALISIS PERSEPSI BRAND ASSOCIATION MENURUT PE...   
2  PENGARUH GAYA KEPEMIMPINAN DEMOKRATIK TERHADAP...   
3  Pengukuran Website Quality Pada Situs Sistem A...   
4  PENGARUH KEPEMIMPINAN DAN KOMPENSASI TERHADAP ...   

                                          abstrak_id  \
0  ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel... 

In [18]:
import pandas as pd

# Load data from pta_manajemen.csv
df = pd.read_csv('pta_manajemen7.csv')

In [19]:
# Display basic information about the dataset
print("Dataset shape:", df.shape)
df


Dataset shape: (1031, 8)


,id,penulis,judul,abstrak_id,abstrak_en,pembimbing_pertama,pembimbing_kedua,prodi
0,80211100070,SATIYAH,PENGARUH FAKTOR-FAKTOR PELATIHAN DAN PENGEMBAN...,"ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel...",ABSTRACT\r\n\r\nIn an effort to increase labor...,"Dra. Hj. S. Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST.SE,M.MT",Manajemen
1,90211200001,Faishal,ANALISIS PERSEPSI BRAND ASSOCIATION MENURUT PE...,Tujuan penelitian ini adalah untuk mengetahui ...,This study wanted to know the brand associatio...,Nurita Andriani,Yustina Chrismardani,Manajemen
2,80211100050,Wahyu Kurniawan,PENGARUH GAYA KEPEMIMPINAN DEMOKRATIK TERHADAP...,NaN,NaN,"Dr. Dra. Hj. Iriani Ismail, MM","Dra. Hj. S. Anugrahini Irawati, MM",Manajemen
3,100211200002,Muhammad Zakaria Utomo,Pengukuran Website Quality Pada Situs Sistem A...,Aplikasi nyata pemanfaatan teknologi informasi...,Academic portal system in University of Trunoj...,"Dr. Ir. Nurita Andriani, MM","Nirma Kurriwati, SP, M.Si",Manajemen
4,80211100044,Hendri Wahyudi Prayitno,PENGARUH KEPEMIMPINAN DAN KOMPENSASI TERHADAP ...,Abstrak\r\nPenelitian ini menggunakan metode k...,Abstract\r\nThis research use quantitative met...,"Dra. Hj. S Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST,SE,.MT",Manajemen
...,...,...,...,...,...,...,...,...
1026,160211100071,Husnul Hotimah,Analisis Cost Volume Profit Untuk Menentukan T...,ABSTRAK\nPenelitian ini bertujuan untuk menget...,ABSTRACT\nThis study aims to determine the cal...,"Hj. Evaliati Amaniyah, S.E., M.S.M.",NaN,Manajemen
1027,160211100291,Uswatun Hasanah,Pengaruh Pelatihan Dan Kompensasi Terhadap Pro...,"ABSTRAK\nUswatun Hasanah, 160211100291, Pengar...","ABSTRACT\nUswatun Hasanah, 160211100291, The E...","Dr. Raden Mas Mochammad Wispandono S.E ., MS",NaN,Manajemen
1028,160211100064,ACH FATHONI,PERAN SERVICE PERFORMANCE DAN CLIMATE ORGANIZA...,ABSTRAK\nTujuan dari penelitian ini adalah unt...,ABSTRACK\n The purpose of this study...,"YUDHI PRASETYA MADA, S.E., M.M.",NaN,Manajemen
1029,160211100030,INTAN YULLIA NINGSIH,BAURAN PROMOSI PADA DEALER YAMAHA TRETAN MOTOR...,ABSTRAK\nPenelitian ini bertujuan: (1) Untuk m...,ABSTRACK\nThis study aims: (1) To find out whe...,"DR. MOHAMMAD ARIEF, S.E., M.M.",NaN,Manajemen


In [20]:
print("\nDataset info:")
df.info()


Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031 entries, 0 to 1030
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  1031 non-null   int64 
 1   penulis             1029 non-null   object
 2   judul               1031 non-null   object
 3   abstrak_id          1026 non-null   object
 4   abstrak_en          1026 non-null   object
 5   pembimbing_pertama  1028 non-null   object
 6   pembimbing_kedua    937 non-null    object
 7   prodi               1031 non-null   object
dtypes: int64(1), object(7)
memory usage: 64.6+ KB


In [21]:
# Tampilkan data "abstrak_id"
df['abstrak_id']

0       ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel...
1       Tujuan penelitian ini adalah untuk mengetahui ...
2                                                     NaN
3       Aplikasi nyata pemanfaatan teknologi informasi...
4       Abstrak\r\nPenelitian ini menggunakan metode k...
                              ...                        
1026    ABSTRAK\nPenelitian ini bertujuan untuk menget...
1027    ABSTRAK\nUswatun Hasanah, 160211100291, Pengar...
1028    ABSTRAK\nTujuan dari penelitian ini adalah unt...
1029    ABSTRAK\nPenelitian ini bertujuan: (1) Untuk m...
1030    Penelitian ini bertujuan untuk dapat mengetahu...
Name: abstrak_id, Length: 1031, dtype: object

## Hapus Missing Value dan Data Duplicat

In [22]:
# Hapus baris dengan Missing Value di 'abstrak_id'
df.dropna(subset=['abstrak_id'], inplace=True)

# Hapus data duplikat
df.drop_duplicates(inplace=True)

## Cleaning

In [23]:
import re

# Fungsi untuk membersihkan teks
def clean_text(text):
    text = text.lower() # Ubah ke huruf kecil
    text = re.sub(r'[^\w\s]', '', text) # Hapus tanda baca
    text = re.sub(r'\d+', '', text) # Hapus nomor
    return text

# Terapkan pembersihan ke kolom 'abstrak_id'
df['cleaned_abstrak_id'] = df['abstrak_id'].apply(clean_text)

# Tampilkan DataFrame
display(df[['abstrak_id', 'cleaned_abstrak_id']].head())

,abstrak_id,cleaned_abstrak_id
0,"ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel...",abstrak\r\nsatiyah pengaruh faktorfaktor pelat...
1,Tujuan penelitian ini adalah untuk mengetahui ...,tujuan penelitian ini adalah untuk mengetahui ...
3,Aplikasi nyata pemanfaatan teknologi informasi...,aplikasi nyata pemanfaatan teknologi informasi...
4,Abstrak\r\nPenelitian ini menggunakan metode k...,abstrak\r\npenelitian ini menggunakan metode k...
5,"Abstrak\r\n\r\nAththaariq, Pengaruh Kompetensi...",abstrak\r\n\r\naththaariq pengaruh kompetensi ...


## Tokenisasi

In [25]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')

# Fungsi untuk melakukan tokenisasi
def tokenize_text(text):
    return word_tokenize(text)

# Terapkan tokenisasi ke kolom 'cleaned_abstrak_id'
df['tokenized_abstrak_id'] = df['cleaned_abstrak_id'].apply(tokenize_text)

# Tampilkan DataFrame dengan kolom hasil tokenisasi
display(df[['cleaned_abstrak_id', 'tokenized_abstrak_id']].head())

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,cleaned_abstrak_id,tokenized_abstrak_id
0,abstrak\r\nsatiyah pengaruh faktorfaktor pelat...,"[abstrak, satiyah, pengaruh, faktorfaktor, pel..."
1,tujuan penelitian ini adalah untuk mengetahui ...,"[tujuan, penelitian, ini, adalah, untuk, menge..."
3,aplikasi nyata pemanfaatan teknologi informasi...,"[aplikasi, nyata, pemanfaatan, teknologi, info..."
4,abstrak\r\npenelitian ini menggunakan metode k...,"[abstrak, penelitian, ini, menggunakan, metode..."
5,abstrak\r\n\r\naththaariq pengaruh kompetensi ...,"[abstrak, aththaariq, pengaruh, kompetensi, do..."


## Stopword Removal

In [26]:
from nltk.corpus import stopwords
nltk.download('stopwords')

# Dapatkan Stop Word bahasa Indonesia
list_stopwords = set(stopwords.words('indonesian'))

# Fungsi untuk menghapus stop words
def remove_stopwords(tokens):
    return [word for word in tokens if word not in list_stopwords]

# Terapkan penghapusan Stop Word ke kolom 'tokenized_abstrak_id'
df['stopwords_removed_abstrak_id'] = df['tokenized_abstrak_id'].apply(remove_stopwords)

# Tampilkan DataFrame
display(df[['tokenized_abstrak_id', 'stopwords_removed_abstrak_id']].head())

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,tokenized_abstrak_id,stopwords_removed_abstrak_id
0,"[abstrak, satiyah, pengaruh, faktorfaktor, pel...","[abstrak, satiyah, pengaruh, faktorfaktor, pel..."
1,"[tujuan, penelitian, ini, adalah, untuk, menge...","[tujuan, penelitian, persepsi, brand, associat..."
3,"[aplikasi, nyata, pemanfaatan, teknologi, info...","[aplikasi, nyata, pemanfaatan, teknologi, info..."
4,"[abstrak, penelitian, ini, menggunakan, metode...","[abstrak, penelitian, metode, kuantitatif, men..."
5,"[abstrak, aththaariq, pengaruh, kompetensi, do...","[abstrak, aththaariq, pengaruh, kompetensi, do..."


## Stemming

In [27]:
!pip install Sastrawi


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [28]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Buat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi untuk melakukan stemming
def stem_tokens(tokens):
    return [stemmer.stem(token) for token in tokens]

# Terapkan stemming ke kolom 'stopwords_removed_abstrak_id'
df['stemmed_abstrak_id'] = df['stopwords_removed_abstrak_id'].apply(stem_tokens)

# Function to find and format stemmed word changes
def get_stemming_changes(original_tokens, stemmed_tokens):
    changes = []
    for original, stemmed in zip(original_tokens, stemmed_tokens):
        if original != stemmed:
            changes.append(f"{original} : {stemmed}")
    return changes

# Dapatkan semua perubahan stemming dari kolom 'stopwords_removed_abstrak_id' dan 'stemmed_abstrak_id'
all_changes = []
for index, row in df.iterrows():
    all_changes.extend(get_stemming_changes(row['stopwords_removed_abstrak_id'], row['stemmed_abstrak_id']))

# Dapatkan perubahan unik dan urutkan
unique_changes = sorted(list(set(all_changes)))

# Cetak perubahan unik
print("Kata-kata yang mengalami stemming:")
for change in unique_changes:
    print(change)

# Tampilkan DataFrame
display(df[['stopwords_removed_abstrak_id', 'stemmed_abstrak_id']].head())

Kata-kata yang mengalami stemming:
acuan : acu
adukkan : aduk
aini : ain
akbari : akbar
akibatnya : akibat
aktivitasnya : aktivitas
alasan : alas
alfi : alf
aliran : alir
analisi : analis
analisisi : analisis
analisisis : analisis
analisisnya : analisis
ancaman : ancam
andari : andar
anggapan : anggap
anggaran : anggar
anggotanya : anggota
angkatan : angkat
angsuran : angsur
apapun : apa
arahan : arah
arahnya : arah
asii : asi
asumsikan : asumsi
atasan : atas
aturan : atur
bahkah : bah
baiknya : baik
bandingkan : banding
bangkalan : bangkal
bangunan : bangun
bantuan : bantu
banyaknya : banyak
batasan : batas
bauran : baur
bawahan : bawah
bawahannya : bawah
bebasnya : bebas
bekasi : bekas
belajar : ajar
belakangi : belakang
belom : bom
beragam : agam
beralamat : alamat
berari : ari
berasal : asal
berbagi : bagi
berbahan : bahan
berbalik : balik
berbasis : bas
berbeda : beda
berbelanja : belanja
berdagangan : dagang
berdampak : dampak
berdampingan : damping
berdasar : dasar
berdasarkan :

,stopwords_removed_abstrak_id,stemmed_abstrak_id
0,"[abstrak, satiyah, pengaruh, faktorfaktor, pel...","[abstrak, satiyah, pengaruh, faktorfaktor, lat..."
1,"[tujuan, penelitian, persepsi, brand, associat...","[tuju, teliti, persepsi, brand, association, l..."
3,"[aplikasi, nyata, pemanfaatan, teknologi, info...","[aplikasi, nyata, manfaat, teknologi, informas..."
4,"[abstrak, penelitian, metode, kuantitatif, men...","[abstrak, teliti, metode, kuantitatif, tekan, ..."
5,"[abstrak, aththaariq, pengaruh, kompetensi, do...","[abstrak, aththaariq, pengaruh, kompetensi, do..."


In [29]:
from collections import Counter

# Join all the stemmed tokens into a single list
all_stemmed_words = [word for tokens in df['stemmed_abstrak_id'] for word in tokens]

# Count the frequency of each word
word_frequencies = Counter(all_stemmed_words)

# Display the most common words and their frequencies
print("Top Most Frequent Words:")
for word, frequency in word_frequencies.most_common():
    print(f"{word}: {frequency}")

Top Most Frequent Words:
pengaruh: 5550
kerja: 5386
teliti: 4381
variabel: 3539
usaha: 2535
signifikan: 2440
uji: 2363
karyawan: 2256
nilai: 1903
hasil: 1781
analisis: 1456
positif: 1307
sampel: 1174
data: 1065
putus: 1040
tuju: 1022
parsial: 996
metode: 965
simultan: 949
tingkat: 927
kunci: 891
regresi: 886
beli: 880
indonesia: 853
pt: 849
puas: 844
t: 776
dasar: 767
ganda: 745
saham: 741
teknik: 739
ratio: 704
sampling: 636
x: 629
return: 626
bursa: 605
milik: 602
efek: 602
daftar: 601
ambil: 600
bimbing: 584
harga: 582
produk: 564
kuantitatif: 542
linier: 539
motivasi: 538
bangkal: 536
dominan: 534
f: 481
oleh: 478
periode: 476
responden: 474
uang: 455
hipotesis: 451
kualitas: 450
konsumen: 436
bawah: 426
pegawai: 426
mm: 420
bank: 413
populasi: 408
kabupaten: 408
r: 407
madura: 407
koefisien: 401
on: 395
to: 392
abstrak: 378
merek: 370
organisasi: 370
equity: 360
purposive: 354
profitabilitas: 351
faktor: 344
studi: 344
thitung: 333
dekat: 332
tbk: 332
prestasi: 330
disiplin: 329
t

In [30]:
# Buat DataFrame baru dengan abstrak asli dan abstrak stemmed
processed_df = df[['abstrak_id', 'stemmed_abstrak_id']].copy()

# Ganti nama kolom
processed_df.rename(columns={'stemmed_abstrak_id': 'hasil_preprocessing'}, inplace=True)

# Konversi frekuensi kata ke DataFrame
frequency_df = pd.DataFrame.from_dict(word_frequencies, orient='index', columns=['frequency'])
frequency_df.index.name = 'word'
frequency_df.sort_values(by='frequency', ascending=False, inplace=True)

# Simpan kedua DataFrame ke lembar yang berbeda dalam file Excel yang sama
with pd.ExcelWriter('processed_data_and_frequencies.xlsx') as writer:
    processed_df.to_excel(writer, sheet_name='Hasil Preprocessing', index=False)
    frequency_df.to_excel(writer, sheet_name='Frekuensi Kata')

print("Hasil preprocessing dan frekuensi kata berhasil disimpan ke dalam file 'processed_data_and_frequencies.xlsx'")

Hasil preprocessing dan frekuensi kata berhasil disimpan ke dalam file 'processed_data_and_frequencies.xlsx'


In [31]:
import pandas as pd

# Baca file Excel
df = pd.read_excel("processed_data_and_frequencies.xlsx")

# Lihat 5 baris pertama
print(df.head())

# Lihat informasi kolom dan tipe datanya
print(df.info())

# Jika ingin menampilkan nama semua kolom
print(df.columns.tolist())


                                          abstrak_id  \
0  ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel...   
1  Tujuan penelitian ini adalah untuk mengetahui ...   
2  Aplikasi nyata pemanfaatan teknologi informasi...   
3  Abstrak\r\nPenelitian ini menggunakan metode k...   
4  Abstrak\r\n\r\nAththaariq, Pengaruh Kompetensi...   

                                 hasil_preprocessing  
0  ['abstrak', 'satiyah', 'pengaruh', 'faktorfakt...  
1  ['tuju', 'teliti', 'persepsi', 'brand', 'assoc...  
2  ['aplikasi', 'nyata', 'manfaat', 'teknologi', ...  
3  ['abstrak', 'teliti', 'metode', 'kuantitatif',...  
4  ['abstrak', 'aththaariq', 'pengaruh', 'kompete...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026 entries, 0 to 1025
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   abstrak_id           1026 non-null   object
 1   hasil_preprocessing  1026 non-null   object
dtypes: object(2)
memory usage: